In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models.models import LstmMseDropout
from utils.data_loader import DataPreperatorPrediction, DataSet
from loss_module import LossModuleMse, LossModuleMle

In [2]:
hyperparam = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path_new" : '../../data/vega_shrinkwrapper_original/NewBlade/', 
        "path_worn" : '../../data/vega_shrinkwrapper_original/WornBlade/',
        "columns" :["timestamp", "cut_torque_target", "cut_lag error_target", 
                    "cut_position_target", "cut_speed_target", "film_position_target", 
                    "film_speed_target", "film_lag_error_target", "cut_torque_predicted", 
                    "cut_lag error_predicted", "cut_position_predicted", "cut_speed_predicted", 
                    "film_position_predicted", "film_speed_predicted", "film_lag_error_predicted", "prediction_loss"]
    },
    "model" : {
        "path" : "../../models/MSE_model/best_model_aws_tanh.pt",
        "input_size" : 7,
        "n_hidden" : 150,
        "sequence_size" : 5,
        "batch_size" : 1,
        "lstm_layer" : 3,
    },
    "filed_location": {
        "prediction_new" : "../visualisation/files/prediction/prediction_new_blade_tanh_seq20.csv",
        "prediction_worn" : "../visualisation/files/prediction/prediction_worn_blade_tanh_seq20.csv",
        "residual_new" : "../visualisation/files/residuals/residual_new_blade_seq5.csv",
        "residual_worn" : "../visualisation/files/residuals/residual_worn_blade_seq5.csv"
    }
}

# Predict new blade data
## Standarize Validation Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from NewBlade Training Data

In [6]:
mean_training_data = [-5.37536613e-02, -2.53111489e-04, -8.82854465e+05, 7.79034183e+02, 1.45531178e+04, 1.37766733e+03, 6.50149764e-01] 
var_training_data = [1.25303578e-01, 1.16898690e-03, 2.86060835e+06, 1.64515717e+06, 6.85728371e+06, 3.63196175e+05, 8.21463343e-03]

### Mean and Variance from Artifical Training Data

In [4]:
mean_training_data= [-5.31764899e-02, -3.98576146e-04, -8.82773455e+05,  8.25672897e+02, 1.47034247e+04,  1.42685595e+03,  6.62155736e-01,  1.23172374e-02]
var_training_data = [1.28792583e-01, 1.21258617e-03, 2.90245238e+06, 1.72279458e+06, 6.83095901e+06, 3.12357562e+05, 3.89033076e-03, 5.01164766e+01]

### Load and scale training data

In [7]:
new_blade_loader = DataPreperatorPrediction(hyperparam['data']['path_new']+'NewBlade001.csv', mean_training_data, var_training_data, hyperparam["model"]["input_size"])
preprocessed_new_blade = new_blade_loader.provide_data()

### Load worn blade dataset and scale them with mean and variance of training data

In [8]:
worn_blade_loader = DataPreperatorPrediction(hyperparam['data']['path_worn']+'WornBlade001.csv', mean_training_data, var_training_data, hyperparam["model"]["input_size"])
preprocessed_worn_blade = worn_blade_loader.provide_data()

In [9]:
dataset_worn_blade = DataSet(preprocessed_worn_blade, timesteps=hyperparam["model"]["sequence_size"])
dataset_new_blade = DataSet(preprocessed_new_blade, timesteps=hyperparam["model"]["sequence_size"])

data_loader_worn_blade = DataLoader(dataset_worn_blade, batch_size=1, num_workers=1, shuffle=False, drop_last=True)
data_loader_new_blade = DataLoader(dataset_new_blade, batch_size=1, num_workers=1, shuffle=False, drop_last=True)

In [10]:
# Initiate and load model
model = LstmMseDropout(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'], dropout_rate=0.2)

PATH = hyperparam["model"]["path"]
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

LstmMseDropout(
  (lstm): LSTM(7, 150, num_layers=3, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=150, out_features=50, bias=True)
  (dropout): Dropout(p=0.2)
  (fc2): Linear(in_features=50, out_features=7, bias=True)
)

In [13]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data.size())

lstm.weight_ih_l0 torch.Size([600, 7])
lstm.weight_hh_l0 torch.Size([600, 150])
lstm.bias_ih_l0 torch.Size([600])
lstm.bias_hh_l0 torch.Size([600])
lstm.weight_ih_l1 torch.Size([600, 150])
lstm.weight_hh_l1 torch.Size([600, 150])
lstm.bias_ih_l1 torch.Size([600])
lstm.bias_hh_l1 torch.Size([600])
lstm.weight_ih_l2 torch.Size([600, 150])
lstm.weight_hh_l2 torch.Size([600, 150])
lstm.bias_ih_l2 torch.Size([600])
lstm.bias_hh_l2 torch.Size([600])
fc1.weight torch.Size([50, 150])
fc1.bias torch.Size([50])
fc2.weight torch.Size([7, 50])
fc2.bias torch.Size([7])


## Predict new blade

In [8]:
# Initiate and load model
model = LstmMseDropout(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'], dropout_rate=0.2)

PATH = hyperparam["model"]["path"]
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create empty dataframe
columns = hyperparam["data"]["columns"]

df = pd.DataFrame(columns=columns)

print("Start predicting")    
##### Predict #####
for batch_number, data in enumerate(data_loader_new_blade):
    
    input_data, target_data = data
    hidden = model.init_hidden()

    # Forward propagation
    output = model(input_data, hidden)
    
    # Calculate loss
    criterion = LossModuleMse(hyperparam["model"]["input_size"], hyperparam["model"]["batch_size"])
    loss = criterion(output, target_data)
    
    # Add values to dataframe 
    output = torch.squeeze(output)
    target_data = torch.squeeze(target_data)
    target_data_np = target_data.data.numpy().tolist()
    predicted_data_np = output.data.numpy().tolist()
    data = [batch_number] + target_data_np + predicted_data_np + [loss.item()]
    df = df.append(pd.Series(data, index=df.columns ), ignore_index=True)

# Save dataframe as csv file
df.to_csv(hyperparam["filed_location"]["prediction_new"], sep=";", index=False)

print("Finished")  

Start predicting


/Users/carlovoss/jupyter_notebooks/masterarbeit/venv_pm/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Finished


## Predict worn blade

In [29]:
# Initiate and load model
model = LstmMseDropout(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'], dropout_rate=0.2)

PATH = hyperparam["model"]["path"]
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create empty dataframe
columns = hyperparam["data"]["columns"]
df = pd.DataFrame(columns=columns)

print("Start predicting")    
##### Predict #####
for batch_number, data in enumerate(data_loader_worn_blade):
    
    input_data, target_data = data
    hidden = model.init_hidden()

    # Forward propagation
    output = model(input_data, hidden)
    
    # Calculate loss
    criterion = LossModuleMse(hyperparam["model"]["input_size"], hyperparam["model"]["batch_size"])
    loss = criterion(output, target_data)
    
    # Add values to dataframe 
    output = torch.squeeze(output)
    target_data = torch.squeeze(target_data)
    target_data_np = target_data.data.numpy().tolist()
    predicted_data_np = output.data.numpy().tolist()
    data = [batch_number] + target_data_np + predicted_data_np + [loss.item()]
    df = df.append(pd.Series(data, index=df.columns ), ignore_index=True)

# Save dataframe as csv file
df.to_csv(hyperparam["filed_location"]["prediction_worn"], sep=";", index=False)

print("Finished") 

Start predicting
Finished


## Compute Residuals for New Blade

In [32]:
data_new_blade = pd.read_csv(hyperparam["filed_location"]["prediction_new"], sep=";")

# Create empty dataframe
columns_residual = ["timestamp", "cut_torque_residuas", "cut_lag_error_residual", "cut_position_residual", "cut_speed_residual", 
           "film_position_residual", "film_speed_residual", "film_lag_error_residual"]
columns_original = data_new_blade.columns
df = pd.DataFrame(columns=columns)

# Compute Residuals
for i in range(1,8):
    df[columns_residual[i]] = data_new_blade[columns_original[i]] - data_new_blade[columns_original[i+7]]

df["timestamp"]=data_new_blade["timestamp"]

# Save dataframe as csv file
df.to_csv(hyperparam["filed_location"]["residual_new"], sep=";", index=False)

## Compute Residuals for New Blade

In [33]:
data_new_blade = pd.read_csv(hyperparam["filed_location"]["prediction_worn"], sep=";")

# Create empty dataframe
columns_residual = ["timestamp", "cut_torque_residuas", "cut_lag_error_residual", "cut_position_residual", "cut_speed_residual", 
           "film_position_residual", "film_speed_residual", "film_lag_error_residual"]
columns_original = data_new_blade.columns
df = pd.DataFrame(columns=columns)

# Compute Residuals
for i in range(1,8):
    df[columns_residual[i]] = data_new_blade[columns_original[i]] - data_new_blade[columns_original[i+7]]

df["timestamp"]=data_new_blade["timestamp"]

# Save dataframe as csv file
df.to_csv(hyperparam["filed_location"]["residual_worn"], sep=";", index=False)